In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 59.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 43.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 39.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:0000:0100:08m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.1 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 5.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import torch
import torch.nn as nn
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
import os
import shutil
from PIL import Image
import cv2

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [12]:
model = InceptionResnetV1(pretrained='vggface2').to(device).eval()

In [13]:
perio_directory = '/kaggle/input/dip-dataset/CrossEyed_Original/PeriCrossEyed'

In [14]:
trans = transforms.Compose([
                transforms.ToPILImage(),
                transforms.Resize((160, 160)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
              ])

In [52]:

# Set the root directory
root_dir = perio_directory


VW_df = None
NIR_df = None
# Loop through the main directories 'L' and 'R'
for img_dir in ['VW', 'NIR']:  
    # Loop through the subject directories\
    a=0
    for main_dir in ['L', 'R']:
        # Loop through the 'VIS' and 'NIR' directories
        for subject_dir in os.listdir(main_dir_path):
            main_dir_path = os.path.join(root_dir, main_dir)  
            subject_dir_path = os.path.join(main_dir_path, subject_dir)
            img_dir_path = os.path.join(subject_dir_path, img_dir)
            
            # Loop through the BMP images and copy them to a destination directory
            img_idx=0
            for img_file in os.listdir(img_dir_path):
                if img_file.endswith(".bmp"):
                    img_idx+=1
                    src_path = os.path.join(img_dir_path, img_file)
                    bmp_image = cv2.imread(src_path)
                    image = trans(bmp_image).to(device).unsqueeze(0)
                    output = model(image).squeeze(0).to('cpu').detach().numpy()
#                     print(output.shape)
                    img_dict = pd.DataFrame({
                        'index': img_idx,
                        'class': a,
                        'embedding': [output]
                    })
#                     print(img_dict)
                    if img_dir=='VW':
                        if VW_df is None:
                            VW_df = img_dict
                        else:
                            VW_df = pd.concat([VW_df, img_dict], ignore_index=True)
                    else:
                        if NIR_df is None:
                            NIR_df = img_dict
                        else:
                            NIR_df = pd.concat([NIR_df, img_dict], ignore_index=True)
            a+=1

In [59]:
embeddings = VW_df['embedding'].values
embeddings_2d = np.stack(embeddings, axis=1)

In [60]:
embeddings_2d = np.transpose(embeddings_2d)

In [61]:
data_VW = pd.DataFrame(embeddings_2d)
data_VW['class'] = VW_df['class'] 

In [63]:
data_VW

,0,1,2,3,4,5,6,7,8,9,...,503,504,505,506,507,508,509,510,511,class
0,-0.000543,-0.038383,-0.115822,0.109722,-0.007099,-0.002594,-0.045077,0.032629,0.000256,-0.035854,...,0.051548,-0.037366,0.019774,0.048183,0.016955,0.006760,-0.000238,0.002287,-0.006563,0
1,0.007925,-0.033022,-0.115046,0.100562,-0.024025,-0.018189,-0.036327,0.029590,0.006668,-0.033554,...,0.040764,-0.059772,0.038549,0.035130,0.013093,0.017712,-0.016275,0.000043,0.016460,0
2,-0.020257,-0.036425,-0.099675,0.123848,-0.011087,-0.006929,-0.043936,0.020392,0.005322,-0.038902,...,0.044783,-0.057322,0.045241,0.051715,0.009900,0.009596,-0.008173,0.015681,-0.006242,0
3,-0.013441,-0.038348,-0.102067,0.128169,-0.016412,-0.012813,-0.046571,0.023529,0.010268,-0.030239,...,0.045848,-0.052456,0.033325,0.041385,0.012202,0.014333,-0.010353,0.009007,0.000563,0
4,-0.003113,-0.027032,-0.106626,0.112149,-0.001122,0.002950,-0.037078,0.025971,-0.001067,-0.025340,...,0.065722,-0.025408,0.028814,0.047617,0.026892,-0.008516,-0.005935,-0.019403,-0.015903,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,-0.041529,-0.041155,-0.101020,0.118733,0.026045,0.035248,-0.045684,0.030973,0.008670,-0.051667,...,0.045692,-0.058080,0.044774,0.081541,0.013997,-0.002458,-0.007737,0.010358,-0.011591,239
1916,-0.019289,-0.076521,-0.101978,0.092160,0.030204,0.009375,-0.046781,0.025203,0.019053,-0.071457,...,0.039507,-0.021375,0.045110,0.073070,0.020128,-0.014391,-0.013720,0.006110,-0.022460,239
1917,-0.032108,-0.054666,-0.086172,0.105993,0.032937,0.009603,-0.053826,0.024157,0.027522,-0.062122,...,0.052971,-0.033823,0.040902,0.072776,0.010315,-0.008697,-0.013759,-0.006897,-0.011100,239
1918,-0.039955,-0.021781,-0.075489,0.118407,0.027112,0.040676,-0.044658,0.025907,0.004368,-0.060948,...,0.041468,-0.072981,0.047607,0.091640,0.017583,-0.001790,-0.017322,0.024238,-0.020799,239


In [64]:
data_VW.to_csv('VIS.csv')